# 配置环境

<ul>
<li><a href='https://git-scm.com/download/win'>下载并安装git</a></li>
<li><a href='https://chromedriver.chromium.org/downloads'>下载版本匹配的chrome webdriver</a></li>
<li>webdriver放到路径中</li>
<li>配置虚拟环境</li>
<li>安装依赖库</li>
</ol>

# 案例：爬取Buff网站饰品价格

In [1]:
#加载需要的库
import bs4
import pandas as pd
import time
import random
from selenium import webdriver

In [2]:
#初始化浏览器&登录
driver = webdriver.Chrome()
driver.get('https://buff.163.com')

In [4]:
#获取所有武器标签
driver.get('https://buff.163.com/market/csgo#tab=buying&page_num=1')
html = driver.execute_script("return document.body.outerHTML;")#
page=bs4.BeautifulSoup(html)
weapon_values=[]
weapons=[]
target=page.find(id="j_h1z1-selType")
for li in target.find_all('li'):
    weapon_values.append(li.get('value'))
    weapons.append(li.text)

In [8]:
#建excel表格存储数据
filename=time.strftime("%Y-%m-%d %H")+'点饰品市场数据.xlsx'
data=pd.DataFrame()
url=r'D:\技术课程\python爬虫\result/'#
data.to_excel(url+filename,sheet_name="空表用于计算")

In [10]:
#分种类爬取饰品
for j in range(len(weapon_values)):
    driver.get('https://buff.163.com/market/csgo#tab=buying&page_num=1&category='+weapon_values[j])
    driver.refresh()
    # HTML from `<body>`
    time.sleep(5+random.random())
    html = driver.execute_script("return document.body.outerHTML;")#
    soup=bs4.BeautifulSoup(html)
    list_html=soup.find('div',id="j_list_card")
    list_href=[]
    list_title=[]
    list_price=[]
    list_need=[]
    list_wearcatogory=[]
    try:
        a_html=list_html.find_all('a')
    except:
        continue
    for i in range(len(a_html)):
        if i%2==0:
            list_href.append("https://buff.163.com/"+a_html[i]['href'])
    for i in range(len(a_html)):
        if i%2==0:
            list_title.append(a_html[i]['title'])
    strong_html=list_html.find_all('strong')
    for strong in strong_html:
        list_price.append(strong.text.replace('¥ ',''))
    span_html=list_html.find_all('span')
    if len(span_html)>len(strong_html):
        for i in range(len(span_html)):
            if i%2 ==0:
                list_need.append(span_html[i].string.replace('\n','').replace(' ','').replace('件需求',''))
            else:
                list_wearcatogory.append(span_html[i].string)
    else:
        for i in range(len(span_html)):
            list_need.append(span_html[i].string.replace('\n','').replace(' ','').replace('件需求',''))
            list_wearcatogory.append('-')

    #存储进表格
    data=pd.DataFrame()
    data['物品']=list_title
    data['Buff求购最高价']=list_price
    data['需求']=list_need
    data['品质']=list_wearcatogory
    data['链接']=list_href

    writer = pd.ExcelWriter(url+filename,mode='a',engine='openpyxl')
    data.to_excel(writer,sheet_name=weapons[j].replace(':',''))
    writer.save()
writer.close()